In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 1. Install the SDK and remove conflicting packages (from notebook cell 5)
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.45.0"

# 2. Import the SDK and some helpers for rendering the output (from notebook cell 6)
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

# 3. Set up a retry helper for API calls (from notebook cell 7)
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

# 4. Set up your API key from Kaggle User Secrets (from notebook cell 8)
#    NOTE: You must have GOOGLE_API_KEY set up in Kaggle Secrets for this to work.
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# 5. Initialize the Gemini API client (from notebook cell 9, before the first prompt)
client = genai.Client(api_key=GOOGLE_API_KEY)

# --- Example of making a basic request (from notebook cell 9) ---
# response = client.models.generate_content(
#     model="gemini-2.0-flash",
#     contents="Explain AI to me like I'm a kid.")
#
# print(response.text)
# Markdown(response.text) # To render as markdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 10.9 MB/s eta 0:00:00


In [7]:
import requests
from bs4 import BeautifulSoup

def get_website_text(url):
    """Fetches a website and extracts its main text content."""
    try:
        response = requests.get(url, timeout=10) # Set a timeout for the request
        response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove script, style, and other non-content elements
        for script in soup(["script", "style", "header", "footer", "nav", "form", "aside"]):
            script.extract()

        # Get text, strip extra whitespace
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)
        return text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None
    except Exception as e:
        print(f"Error parsing content from {url}: {e}")
        return None

# --- Main part: Using the fetched content in a prompt ---

# 1. Define the website you want the AI to "read"
website_url = ""
print(f"Fetching content from: {website_url}\n")

# 2. Use your Python code to fetch and extract text
website_content = get_website_text(website_url)

if website_content:
    # 3. Truncate content if it's too long for the model's context window
    # Gemini 2.0 Flash has an input_token_limit of 1,048,576 tokens.
    # While that's very large, for shorter prompts or to be safe, you might truncate.
    # For simplicity, we'll use a character limit here, but a token count is more accurate.
    max_chars_for_context = 100000 # Adjust based on your model's token limits and prompt structure
    if len(website_content) > max_chars_for_context:
        website_content = website_content[:max_chars_for_context] + "\n... [Content truncated due to length]"

    # 4. Construct the prompt including the website content and your query
    query = "List the features of each product in the catalog one paragraph for each product"
    full_prompt = f"""
    Here is some information from a website:
    ---
    {website_content}
    ---

    Based ONLY on the provided text, {query}
    """

    print(f"Sending query to AI (content length: {len(website_content)} chars)...\n")

    # 5. Send the prompt to the Gemini API
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=full_prompt
    )

    # 6. Display the AI's response
    display(Markdown(response.text))
else:
    print("Could not retrieve website content to send to the AI.")

Fetching content from: https://www.reacthealth.com/catalog

Error fetching URL https://www.reacthealth.com/catalog: HTTPSConnectionPool(host='www.reacthealth.com', port=443): Max retries exceeded with url: /catalog (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f35e489f1d0>, 'Connection to www.reacthealth.com timed out. (connect timeout=10)'))
Could not retrieve website content to send to the AI.


In [4]:
 query = "Write ddown the salient features of Alim Alimah Onsite and Online course as well as about some teachers."
full_prompt = f"""
            Here is some information from a website:
            ---
            {website_content}
            ---

            Based ONLY on the provided text, {query}
            """

print(f"Sending query to AI (content length: {len(website_content)} chars)...\n")

    # 5. Send the prompt to the Gemini API
response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=full_prompt
    )

    # 6. Display the AI's response
display(Markdown(response.text))



Sending query to AI (content length: 9215 chars)...



Okay, here's a breakdown of the Alim Alimah courses and teacher information based solely on the provided text:

**Alim Alimah Online Course:**

*   **Description:** Dive deep into Islamic knowledge from the comfort of your home.
*   **Content:** Structured learning in Tafseer, Hadith, Fiqh, and more.
*   **Instruction:** Guided by expert scholars.

**Alim Alimah Onsite Course:**

*   **Description:** Experience the rich tradition of Islamic education with face-to-face learning.
*   **Format:** Provides immersive studies in key Islamic sciences.
*   **Location:** Held at their campus.

**Information about Some Teachers:**

*   **Sheikh Abu Umar:** Graduate, University of Madinah, Faculty of Hadith, Saudi Arabia. Instructor
*   **Sheikh Muhammad Shareef Al-Madani:** Graduate University of Madinah, Instructor Teacher at Jamia Abi Bakar al Islamia. Instructor
*   **Sheikh Zia Ul Haq Bhatti:** Graduated Jamia Sattaria Islamia, Vice Sheikh Ul-Hadith Mahad as Salafi. Instructor
*   **Sheikh Ubaid-ur-Rehman:** Graduate Mahad as Salafi, Senior Teacher at Mahad as Salafi. Instructor
*   **Sheikh Nizaam-ud-Din:** Graduate Madinah University, Instructor at Mahad as Salafi. Instructor
*   **Sheikh Abdus Samad:** Graduated Jamia Islamia Madinah Munawwara, Mahad as Salafi. Instructor
*   **Sheikh Taha Pasha:** CEO & Instructor, Founding Member Burooj Institute, Graduated from Mahad as Salafi and Jamia Abi Bakr al Islamia
*   **Sheikh Hunain Habib:** Graduate Mahad as Salafi, CAF and ACCA certified. Instructor
*   **Sheikh Muhammad Adeel Hanif:** Instructor | Academics Head, Founding Member Burooj Institute, Graduated Jamia Abi Bakr
*   **Sheikh Mughees Ur Rahman:** Head Research Department, Instructor Mahad as Salafi | Graduated Mahad as Salafi | Specialization in Hadith
*   **Ustad Muhammad Zain:** Instructor, Graduate Burooj Institute, Mechanical Engineer - GIK

